In [7]:
cd

/home/ubuntu


In [8]:
cd cw/OneFormer/

/home/ubuntu/cw/OneFormer


RectLabel Labelme json format -> PanopticFCN json

In [9]:
import json
import numpy as np
import os

input_path = "/home/ubuntu/cw/OneFormer/input/Labelme/*"
output_path = "/home/ubuntu/cw/OneFormer/output/Polygon/*"
os.makedirs(output_path, exist_ok=True)

img_files = os.listdir(input_path)
img_files.sort()

for filename in os.listdir(input_path):
    if filename.endswith(".json"):
        input_filepath = os.path.join(input_path, filename)
        with open(input_filepath, "r") as f:
            data = json.load(f)
            
            # 1 field
            objects = []
            num_instances = len(data['shapes'])
   
            for i in range(num_instances):
                # Create objects
                object_field = {
                    'label' : data['shapes'][i]['label'],
                    'polygon' : data['shapes'][i]['points']
                }
                objects.append(object_field)
                
            # Create FCN JSON dictionary
            fcn_dict = {}
            fcn_dict['imgHeight'] = data['imageHeight']
            fcn_dict['imgWidth'] = data['imageWidth']
            fcn_dict['objects'] = objects
            
            name, ext = filename.split('leftImg8bit')
            output_filename = name + "gtFine_polygons" + ext
            output_filepath = os.path.join(output_path, output_filename)
            
            with open(output_filepath, "w", encoding="utf-8") as fo:
                json.dump(fcn_dict, fo)
            
            print(f"Output FCN Input JSON saved at {output_filepath}")
            
        # 첫 번째 파일만 처리    
        # break
        

Output FCN Input JSON saved at output_json/FCN_json2/frankfurt_000000_017228_gtFine_polygons.json
Output FCN Input JSON saved at output_json/FCN_json2/frankfurt_000000_007365_gtFine_polygons.json
Output FCN Input JSON saved at output_json/FCN_json2/frankfurt_000000_004617_gtFine_polygons.json
Output FCN Input JSON saved at output_json/FCN_json2/frankfurt_000001_017842_gtFine_polygons.json
Output FCN Input JSON saved at output_json/FCN_json2/frankfurt_000001_040575_gtFine_polygons.json
Output FCN Input JSON saved at output_json/FCN_json2/frankfurt_000000_006589_gtFine_polygons.json
Output FCN Input JSON saved at output_json/FCN_json2/frankfurt_000001_005410_gtFine_polygons.json
Output FCN Input JSON saved at output_json/FCN_json2/frankfurt_000001_062793_gtFine_polygons.json
Output FCN Input JSON saved at output_json/FCN_json2/frankfurt_000001_005703_gtFine_polygons.json
Output FCN Input JSON saved at output_json/FCN_json2/frankfurt_000001_042733_gtFine_polygons.json
Output FCN Input JSO

RectLabel COCO json format -> PanopticFCN val json

In [ ]:
import collections
import json
import numpy as np
import os

input_files = "output_json/cityscapes_panoptic_val.json"
os.makedirs(output_path, exist_ok=True)

with open(input_files, "r") as f:
    data = json.load(f)
    categories = data['categories']

In [12]:
import collections
import json
import numpy as np
import os

input_files = "output_json/coco.json"
output_path = "output_json/FCN_json/"
os.makedirs(output_path, exist_ok=True)


with open(input_files, "r") as f:
    data = json.load(f)
    # image_id를 기준으로 묶어줌
    annotations = data['annotations']
    annoimages = data['images']
    images_instances = len(data['images'])
    annotation_field = {}
    image_field = {}
    images = []

    for ann in annotations:
        image_id = ann['image_id']
        cateid = ann['category_id']
        mapping = {8: 7, 9: 8, 12: 11, 13: 12, 14: 13, 18: 17, 20: 19, 21: 20, 22: 21, 23: 22, 24: 23, 25: 24, 26: 25, 27: 26, 28: 27, 29: 28, 32: 31, 33: 32 }
        cateid = mapping.get(cateid, 255)
            
        if image_id not in annotation_field:
            for annimg in annoimages:
                if annimg['id'] == ann['image_id']:
                    name, ext = annimg['file_name'].split('_leftImg8bit')
                    annotation_field[image_id] = {
                        'file_name' : annimg['file_name'],
                        'image_id' : name,
                        'segment_info' : []
                    }
        seg_info = {
            'id': ann['id'],
            'category_id' : cateid,
            'area': ann['area'],
            'bbox': ann['bbox'],
            'iscrowd': ann['iscrowd'],
        }
        annotation_field[image_id]['segment_info'].append(seg_info)

    annotations = list(annotation_field.values())
    
    for j in range(images_instances):
        name, ext = data['images'][j]['file_name'].split('_leftImg8bit')
        image_field = {
            'file_name' : data['images'][j]['file_name'],
            'id' : name,
            'height' : data['images'][j]['height'],
            'width' : data['images'][j]['width']
        }
        images.append(image_field)
    
    fcn_val_dict = {}
    fcn_val_dict['annotations'] = annotations
    fcn_val_dict['categories'] = categories
    fcn_val_dict['images'] = images
    
    output_filename = "chae_cityscapes_panoptic_final_2.json"
    output_filepath = os.path.join(output_path, output_filename)

    with open(output_filepath, "w", encoding="utf-8") as f:
        json.dump(fcn_val_dict, f)

    print(f"Output coco json saved at {output_filepath}")

Output coco json saved at output_json/FCN_json/chae_cityscapes_panoptic_final_2.json
